In [63]:
from pyspark.sql import SparkSession, functions as fn
from pyspark.sql.types import *

In [4]:
spark =  SparkSession.builder.appName("practic3_16_4_2024").getOrCreate()

24/04/18 16:27:15 WARN Utils: Your hostname, Venugopals-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.220.233.159 instead (on interface en0)
24/04/18 16:27:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 16:27:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/04/18 16:27:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [75]:
schema  = ['name','age']
df = spark.createDataFrame(data = [('James|Olson',24),('Jeremy|Clarkson',69)]
                           ,schema = schema)


In [76]:
df.show()

+---------------+---+
|           name|age|
+---------------+---+
|    James|Olson| 24|
|Jeremy|Clarkson| 69|
+---------------+---+



TypeError: 'Column' object is not callable

In [82]:
split_name = fn.udf(lambda x:x.split("|"),ArrayType(StringType()))
# split_col = fn.split(df['name'],'\\|')
df = df.withColumn('firstName',split_name(df["name"]).getItem(0))\
    .withColumn('lastName',split_name("name").getItem(1))
# df.withColumn('lastName',split_name[df['name']][1])
df.show()

+---------------+---+---------+--------+
|           name|age|firstName|lastName|
+---------------+---+---------+--------+
|    James|Olson| 24|    James|   Olson|
|Jeremy|Clarkson| 69|   Jeremy|Clarkson|
+---------------+---+---------+--------+



In [114]:
@fn.udf(returnType = StringType())
def add(x):
    return x + "__"

# a = fn.udf(add,StringType())
# @add_
# def change(x)->str:
#     return lambda x:x + "--"

# df.withColumn("exp",add(fn.col("firstName"))).show()
df.withColumn("exp",add(fn.col("firstName"))).show()


+---------------+---+---------+--------+--------+
|           name|age|firstName|lastName|     exp|
+---------------+---+---------+--------+--------+
|    James|Olson| 24|    James|   Olson| James__|
|Jeremy|Clarkson| 69|   Jeremy|Clarkson|Jeremy__|
+---------------+---+---------+--------+--------+



#### 101 Pyspark exercises

How to import Pyspark and check the version

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


In [2]:
spark = SparkSession.builder.appName('pyspark_questions').master('local[1]').getOrCreate()

24/04/18 16:41:18 WARN Utils: Your hostname, Venugopals-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.220.233.159 instead (on interface en0)
24/04/18 16:41:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 16:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51874)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pyspark_practice/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/envs/pyspark_practice/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/envs/pyspark_practice/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/envs/pyspark_practice/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/opt/anaconda3/envs/pyspark_practice/lib/python3.12/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/anaconda3/envs/pyspark_practice/lib/python3.12/site-packages/pyspark/accumulators.py", l

In [33]:
spark.version

'3.5.1'

In [125]:
df = spark.createDataFrame(data = [
    ('a','b','c')
    ,('d','e','f')
])

In [142]:
df=  df.select('*').withColumn('index',row_number().over(Window.orderBy(monotonically_increasing_id())))

In [143]:
df.show()

24/04/18 15:47:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/18 15:47:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/18 15:47:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+---+---+-----+
| _1| _2| _3|index|
+---+---+---+-----+
|  a|  b|  c|    1|
|  d|  e|  f|    2|
+---+---+---+-----+



In [3]:
list1 = [
    ('a','b','c')
    ,('d','e','f')
]
list2 = [
    ('g','h','i')
    ,('j','k','l')
]


In [4]:
rdd1 = spark.sparkContext.parallelize(list1)
rdd2 = spark.sparkContext.parallelize(list2)

In [5]:
rdd1.getNumPartitions()

1

24/04/18 16:41:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


UnionRDD[295] at union at NativeMethodAccessorImpl.java:0

In [160]:
df1 = spark.createDataFrame(rdd1+rdd2)

In [161]:
df1.show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
|  a|  b|  c|
|  d|  e|  f|
|  g|  h|  i|
|  j|  k|  l|
+---+---+---+



In [163]:
#get items of list a not in list b

In [162]:
list_a = [1,2,3,4,5]
list_b = [4,5,6,7,8]

In [164]:
rdd3 = spark.sparkContext.parallelize(list_a)
rdd4 = spark.sparkContext.parallelize(list_b)

In [170]:
vals = rdd3.subtract(rdd4)
print(vals.collect())

[1, 2, 3]


In [171]:
# get items not common to both list a and list b

In [174]:
rdd3.collect()
rdd4.collect()

[4, 5, 6, 7, 8]

In [176]:
rdd3.union(rdd4).subtract(rdd3.intersection(rdd4)).collect()

[1, 2, 3, 6, 7, 8]

In [179]:
rdd3+rdd4

UnionRDD[371] at union at NativeMethodAccessorImpl.java:0

In [189]:
import numpy as np

In [206]:
l = rdd3.union(rdd4)

# l = l.collect()
# print(np.min(l))
# print(np.max(l))
# print(np.median(l))
# print(np.percentile(l,25))
# print(np.percentile(l,75))
# # print(l.med())
# print(l.percentile(25))

In [207]:
df_num = spark.createDataFrame(l.collect(),IntegerType())

In [208]:
df_num.describe().show()

+-------+-----------------+
|summary|            value|
+-------+-----------------+
|  count|               10|
|   mean|              4.5|
| stddev|2.173067468400883|
|    min|                1|
|    max|                8|
+-------+-----------------+



In [209]:
df_num.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
|    4|
|    5|
|    6|
|    7|
|    8|
+-----+



In [1]:
#get frequency counts of unique items of a column

In [6]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()


/opt/anaconda3/envs/pyspark_practice/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=48858) is multi-threaded, use of fork() may lead to deadlocks in the child.


+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+



In [7]:
df.groupBy(col('name'),col('job')).count().show()
df_freq = df.groupBy(col('job')).count().orderBy('count',ascending = False)

+----+---------+-----+
|name|      job|count|
+----+---------+-----+
| Sam|   Doctor|    1|
|Mary|Scientist|    1|
|John| Engineer|    2|
| Bob| Engineer|    2|
| Bob|Scientist|    1|
+----+---------+-----+



In [38]:
jobs = df_freq.limit(2).select('job')

In [59]:
jobs = set([i.job for i in jobs.select('job').collect()])

In [61]:
df = df.withColumn('job',when(col('job').isin(jobs), col('job')).otherwise('Other'))

In [62]:
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



24/04/18 19:00:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 963184 ms exceeds timeout 120000 ms
24/04/18 19:00:22 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/18 19:15:36 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o